In [93]:
import dotenv, os
import csv
from openai import OpenAI

In [94]:
dotenv.load_dotenv(".env")
print(os.getenv("OPENAI_API_KEY"))

sk-proj-ooeZThAsO3CjYCoBKQtvT3BlbkFJlvwUmwpAuvMjGj5lC7Sx


In [95]:
MODEL="gpt-4o-mini"
client = OpenAI()

In [96]:
def get_orders():
  with open(os.getcwd()+'/data/orders.csv', mode='r') as infile:
    reader = csv.reader(infile)
    next(reader, None)
    # create orders dict with key first column 
    orders = {str(row[0]):[row[1], row[2]] for row in reader}
    
  # print(orders)
  return orders
  
def get_order_info_by_number(number:str):
  orders = get_orders()
  try:
    return str(orders[str(number)][0])
  except KeyError:
    return "NA"
  
# get_order_info_by_number('3004')

In [97]:
 
assistant = client.beta.assistants.create(
  instructions="You are a e-shop order assistant. You can provide information about user's orders",
  model=MODEL,
  tools=[
    {
      "type": "function",
      "function": {
        "name": "get_order_info",
        "description": "Get the order status",
        "parameters": {
          "type": "object",
          "properties": {
            "order_number": {
              "type": "string",
              "description": "The order number containing 4 digits and staring from 3"
            }
          },
          "required": ["order_number",]
        }
      }
    },
  ]
)

In [98]:
thread = client.beta.threads.create()

In [111]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="What is the status of my order 1876238762401?",
)

In [112]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [113]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_hhUVZE9a0e4FCm4bNZ4EDbmy', assistant_id='asst_Qo8j9N72VyiC0Y000Y6aZjN1', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='It looks like the order number you provided is not valid. Please check and provide an order number that is 4 digits long and starts with the number 3.'), type='text')], created_at=1721988793, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_lba5v9U0MgUinrTthZFvjDEd', status=None, thread_id='thread_6G7wbcR97kTbLXKpAr4noQWx'), Message(id='msg_u0pA7mXPrJK2g8r7iaP85RU8', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the status of my order 1876238762401?'), type='text')], created_at=1721988790, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_6G7wbcR97kTbL

In [108]:
# Define the list to store tool outputs
tool_outputs = []
 
# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
  if tool.function.name == "get_order_info":
    
    kargs = eval(tool.function.arguments)
    print(kargs)
    
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": get_order_info_by_number(kargs['order_number'])
    })
      
# print(tool_outputs)

{'order_number': '3001'}


In [109]:
# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")
 
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

Tool outputs submitted successfully.
SyncCursorPage[Message](data=[Message(id='msg_UxniNtxdjLgkPdql28KZPtjU', assistant_id='asst_Qo8j9N72VyiC0Y000Y6aZjN1', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Your order 3001 has been delivered.'), type='text')], created_at=1721988769, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_R2tGBj5Lq4cRp6bkwqDQxna1', status=None, thread_id='thread_6G7wbcR97kTbLXKpAr4noQWx'), Message(id='msg_AwZ6KWrmeF8aBbnQPHIRd7Er', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the status of my order 3001?'), type='text')], created_at=1721988760, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_6G7wbcR97kTbLXKpAr4noQWx'), Message(id='msg_2sk6cfPBX2zg6Vr1cTBbGpx5', assistant_id='asst_Qo8j9N72Vy

In [110]:
print(messages.data[0].content[0].text.value)

Your order 3001 has been delivered.
